# 신경망 깊게 쌓아 컬러 데이터셋에 적용하기
Convolutional Neural Network (CNN) 을 쌓아올려 딥한 러닝을 해봅시다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

## 하이퍼파라미터 

In [3]:
EPOCHS     = 300
BATCH_SIZE = 128

## 데이터셋 불러오기

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32, padding=4),
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified


## ResNet 모델 만들기

In [5]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

## 준비

In [6]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [7]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

## 학습하기

In [8]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

## 테스트하기

In [9]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 훈련이 되는지 확인해봅시다!

In [10]:
for epoch in range(1, EPOCHS + 1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

[1] Test Loss: 1.4414, Accuracy: 46.96%
[2] Test Loss: 1.6129, Accuracy: 47.86%
[3] Test Loss: 1.5045, Accuracy: 52.38%
[4] Test Loss: 1.2362, Accuracy: 60.62%
[5] Test Loss: 0.8269, Accuracy: 71.20%
[6] Test Loss: 0.8386, Accuracy: 71.09%
[7] Test Loss: 1.0024, Accuracy: 68.15%
[8] Test Loss: 0.7147, Accuracy: 76.10%
[9] Test Loss: 0.8604, Accuracy: 70.36%
[10] Test Loss: 0.8569, Accuracy: 71.94%
[11] Test Loss: 1.1698, Accuracy: 64.97%
[12] Test Loss: 1.5368, Accuracy: 61.46%
[13] Test Loss: 1.0091, Accuracy: 68.85%
[14] Test Loss: 0.7794, Accuracy: 74.58%
[15] Test Loss: 0.7072, Accuracy: 76.37%
[16] Test Loss: 0.8953, Accuracy: 70.61%
[17] Test Loss: 0.7582, Accuracy: 75.88%
[18] Test Loss: 0.7556, Accuracy: 75.05%
[19] Test Loss: 0.6844, Accuracy: 76.41%
[20] Test Loss: 0.6182, Accuracy: 78.93%
[21] Test Loss: 0.6343, Accuracy: 78.74%
[22] Test Loss: 0.7613, Accuracy: 75.33%
[23] Test Loss: 0.8388, Accuracy: 72.89%
[24] Test Loss: 0.8615, Accuracy: 73.75%
[25] Test Loss: 0.8367, A

[199] Test Loss: 0.3071, Accuracy: 91.11%
[200] Test Loss: 0.3058, Accuracy: 91.04%
[201] Test Loss: 0.3044, Accuracy: 91.02%
[202] Test Loss: 0.3075, Accuracy: 91.02%
[203] Test Loss: 0.3071, Accuracy: 90.97%
[204] Test Loss: 0.3071, Accuracy: 90.99%
[205] Test Loss: 0.3064, Accuracy: 91.04%
[206] Test Loss: 0.3072, Accuracy: 91.02%
[207] Test Loss: 0.3032, Accuracy: 91.03%
[208] Test Loss: 0.3071, Accuracy: 91.03%
[209] Test Loss: 0.3022, Accuracy: 91.18%
[210] Test Loss: 0.3029, Accuracy: 91.14%
[211] Test Loss: 0.3062, Accuracy: 91.21%
[212] Test Loss: 0.3042, Accuracy: 91.05%
[213] Test Loss: 0.3073, Accuracy: 91.00%
[214] Test Loss: 0.3039, Accuracy: 91.13%
[215] Test Loss: 0.3068, Accuracy: 91.05%
[216] Test Loss: 0.3042, Accuracy: 91.05%
[217] Test Loss: 0.3063, Accuracy: 91.00%
[218] Test Loss: 0.3066, Accuracy: 90.97%
[219] Test Loss: 0.3053, Accuracy: 91.07%
[220] Test Loss: 0.3068, Accuracy: 91.12%
[221] Test Loss: 0.3055, Accuracy: 91.13%
[222] Test Loss: 0.3032, Accuracy: